## Strategy Summary

This notebook follows an optimal strategy for preprocessing customer-agent dialogues for sentiment classification:

- Retains `customer:` and `agent:` tags to preserve speaker roles.
- Removes generic agent greetings to avoid introducing bias.
- Strips noise: punctuation, emails, phone numbers, URLs, and excess whitespace.
- Removes known polite/boilerplate phrases that add no sentiment signal.
- Fixes common spelling issues that harm tokenization.
- Splits the dataset using stratified sampling to balance sentiment classes.
        

In [8]:
import pandas as pd
import re
import string
import os
from sklearn.model_selection import train_test_split

In [9]:
# Load dataset and encode labels
df = pd.read_csv("data/train.csv")[["customer_sentiment", "conversation"]].dropna()
sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2}
df['label'] = df['customer_sentiment'].map(sentiment_map)

In [ ]:

# Oversample minority class (label=2) before train/test split
from sklearn.utils import resample

df_major = df[df['label'].isin([0, 1])]
df_minor = df[df['label'] == 2]

df_minor_upsampled = resample(df_minor,
                              replace=True,
                              n_samples=400,  # Match approx. label 0
                              random_state=42)

df = pd.concat([df_major, df_minor_upsampled])
print("✅ Oversampled class 2. New label distribution:")
print(df['label'].value_counts())


✅ Oversampled class 2. New label distribution:
label
1    542
0    411
2    400
Name: count, dtype: int64


In [11]:
custom_phrases = [
    r'\bis there anything else i can (assist|help) you with\b',
    r'\bthank you for choosing brownbox\b',
    r'\bthank you\b',
    r'\byoure welcome\b',
    r'\btake care\b',
    r'\bgoodbye\b',
    r'\bplease\b',
    r'\bthanks\b',
    r'\bsure\b',
    r'\bno thats all\b',
    r'\bhave a (nice|great|good) day\b',
    r'\bappreciate\b',
    r'\bfor contacting brownbox customer support\b'
]

misspellings = {
    'ts': 'this', 'witn': 'within', 'anytng': 'anything',
    'ithis': 'it has', 'thathis': 'that is', 'as you': 'assure you',
    'en that': 'ensure that'
}

def clean_conversation(text):
    text = text.lower()
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\+?\d[\d\s\-().]{8,}\d', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)

    lines = text.strip().split('\n')

    if lines and lines[0].startswith("agent:"):
        if any(greet in lines[0] for greet in [
            "thank you for calling", "hi", "hello", "this is", "my name is",
            "how can i help you", "how may i assist you"
        ]):
            lines = lines[1:]

    text = ' '.join(lines)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()

    for phrase in custom_phrases:
        text = re.sub(phrase, '', text)

    for wrong, right in misspellings.items():
        text = text.replace(wrong, right)

    # Remove "customer" or "agent" if they are at the end of the line
    text = re.sub(r'\b(customer|agent)\b\s*$', '', text)

    return text

In [12]:
# Apply cleaning function
df['conversation'] = df['conversation'].astype(str).apply(clean_conversation)

In [13]:
# Stratified train/validation split
train_df, val_df = train_test_split(
    df[['conversation', 'label']], test_size=0.2, stratify=df['label'], random_state=42
)

In [14]:
# Save output
os.makedirs("data", exist_ok=True)
train_df.to_csv("data/train_cleaned.csv", index=False)
val_df.to_csv("data/val_cleaned.csv", index=False)

# Check class distributions
print("📊 Train set distribution:")
print(train_df['label'].value_counts())  # actual count
print(train_df['label'].value_counts(normalize=True).round(3))  # proportions

print("\n📊 Validation set distribution:")
print(val_df['label'].value_counts())
print(val_df['label'].value_counts(normalize=True).round(3))

📊 Train set distribution:
label
1    433
0    329
2    320
Name: count, dtype: int64
label
1    0.400
0    0.304
2    0.296
Name: proportion, dtype: float64

📊 Validation set distribution:
label
1    109
0     82
2     80
Name: count, dtype: int64
label
1    0.402
0    0.303
2    0.295
Name: proportion, dtype: float64
